<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [148]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [149]:
# FIXME
df1=pd.read_csv(archivos_anio[0])
df2=pd.read_csv(archivos_anio[1])

#normalizar columnas
df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

#juntar df1 y df2
df_anio = pd.concat([df1,df2])
df_anio.head()


,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [150]:
#parte b
df_codigos['codigo_iso'].value_counts()

,count
codigo_iso,
ZWE,2
AFG,1
ALB,1
AGO,1
ARE,1
...,...
VEN,1
WSM,1
YEM,1


In [151]:
#
df_codigos[df_codigos["codigo_iso"]=="ZWE"]

,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


In [152]:
#eliminar los repetidos de ZWE igual a malo
df_codigos = df_codigos[df_codigos['pais'] != 'malo']
df_codigos['codigo_iso'].value_counts()

,count
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1


In [153]:
#paises repetidos
df_codigos['pais'].value_counts()

,count
pais,
Nigeria,2
Angola,1
Afghanistán,1
Andorra,1
Emiratos Árabes Unidos,1
...,...
Samoa,1
Yemen,1
Sudáfrica,1


In [154]:
#
df_codigos[df_codigos["pais"]=="Nigeria"]

,codigo_iso,pais
118,NER,Nigeria
119,NGA,Nigeria


In [155]:
df_codigos = df_codigos[df_codigos['codigo_iso'] != 'NGA']
df_codigos['pais'].value_counts()

,count
pais,
Afghanistán,1
Angola,1
Albania,1
Andorra,1
Emiratos Árabes Unidos,1
...,...
Samoa,1
Yemen,1
Sudáfrica,1


In [156]:
#parte c
df = pd.merge(df_anio, df_codigos, on='codigo_iso')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [157]:
#Estructura del DataFrame
#filas y columnas
df.shape

(3043, 5)

Hay 3.043 filas y 5 columnas

In [158]:
#¿Cuáles son los nombres de las columnas?
df.columns

Index(['codigo_iso', 'anio', 'indice', 'ranking', 'pais'], dtype='object')

Los nombres de las columnas son codigo_iso, anio, indice, ranking y pais

In [159]:
#¿Qué tipo de datos tiene cada columna?
df.dtypes

,0
codigo_iso,object
anio,int64
indice,float64
ranking,float64
pais,object


Las columna codigo_iso es de tipo object\
La columna anio es de tipo int64\
La columna indice es de tipo float64\
La columna ranking es de tipo float64\
La columna pais es de tipo object

No hay columnas con algun tipo de datos inesperados.

In [160]:
#Resumen estadístico
#Genera un resumen estadístico del conjunto de datos con .describe(). ¿Qué observas sobre los valores de indice y ranking?
df.describe()

,anio,indice,ranking
count,3043.000000,2648.000000,2820.000000
mean,2009.941176,206.801866,480.281915
std,5.786029,2703.627558,6494.358133
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.250000,34.000000
50%,2009.000000,27.935000,70.000000
75%,2015.000000,41.262500,110.000000
max,2019.000000,64536.000000,121056.000000


En promedio se tiene aproximadamante un indice de libertad de $206.8$ y un ranking promedio de $480.3$, donde hay un indice minimo de $0$ y un ranking minimo de $1$, ademas el indice mas grande es de $64536$ y un ranking maximo de $121056$.

Ademas, hay datos nulos pues el inidce tiene un total de $2648$ y el ranking un total de $2820$, para un total de 3043 filas. Tambien, se tiene que el $75\%$ de los datos son menores a $41.3$ en el indice y $110$ en el ranking, por tanto la gran mayoria de datos se concentra en valores bajos, sin embargo el promedio del inidce y el ranking son mayores a estos numeros respectivamente, esto quiere decir que hay numeros que son muy grandes que afectan al promedio, como el valor maximo del indice y del ranking.

In [161]:
#¿Qué valores mínimo, máximo y promedio tiene la columna indice?
min=df['indice'].min()
max=df['indice'].max()
mean=df['indice'].mean()
print("El valor minimo del indice es:",min)
print("El valor maximo del indice es:",max)
print("El valor promedio del indice es:",mean)

El valor minimo del indice es: 0.0
El valor maximo del indice es: 64536.0
El valor promedio del indice es: 206.80186555891243


In [162]:
#¿Qué países presentan los valores extremos en indice y ranking?
df[df['indice']==min]
pmin=df[df['indice']==min]
pmax=df[df['indice']==max]
rmin=df[df['ranking']==1]
rmax=df[df['ranking']==121056]
print("Los paises que presentan el valor minimo del indice son:")
print(pmin['pais'])
print("Los paises que presentan el valor maximo del indice son:")
print(pmax['pais'])
print("Los paises que presentan el valor minimo del ranking son:")
print(rmin['pais'])
print("Los paises que presentan el valor maximo del ranking son:")
print(rmax['pais'])

Los paises que presentan el valor minimo del indice son:
1297       Dinamarca
1306       Finlandia
1328         Irlanda
1374         Noruega
1404          Suecia
1460           Suiza
1485       Finlandia
1510        Islandia
1552    Países Bajos
1553         Noruega
1583          Suecia
Name: pais, dtype: object
Los paises que presentan el valor maximo del indice son:
2058    Kosovo
Name: pais, dtype: object
Los paises que presentan el valor minimo del ranking son:
53          Finlandia
78           Islandia
120      Países Bajos
121           Noruega
232         Finlandia
257          Islandia
299      Países Bajos
300           Noruega
386             Suiza
402         Dinamarca
411         Finlandia
433           Irlanda
436          Islandia
478      Países Bajos
479           Noruega
507        Eslovaquia
565             Suiza
581         Dinamarca
590         Finlandia
612           Irlanda
615          Islandia
657      Países Bajos
658           Noruega
769         Finlandia
79

In [163]:
#Datos faltantes
#¿Cuántos valores nulos hay en cada columna?
df.isnull().sum()

,0
codigo_iso,0
anio,0
indice,395
ranking,223
pais,0


Solo en indice y en ranking hay 395 y 223 valores nulos respectivamente.

In [164]:
#¿Qué proporción de observaciones tienen valores faltantes?
(df.isnull().sum()/df.shape[0])*100

,0
codigo_iso,0.000000
anio,0.000000
indice,12.980611
ranking,7.328294
pais,0.000000


Para el indice representa un $12.98\%$ de valores faltantes y para el ranking representa un $7.33$ de valores faltantes.

In [165]:
#¿Hay columnas con más del 30% de datos faltantes?
(df.isnull().sum()/df.shape[0])*100>30

,0
codigo_iso,False
anio,False
indice,False
ranking,False
pais,False


No existen columnas con mas del $30\%$ de datos faltantes.

In [166]:
#Unicidad y duplicados
#¿Cuántos países distintos (pais) hay en el DataFrame?
df['pais'].nunique()

179

Hay $179$ paises distintos en el DataFrame.

In [167]:
#¿Cuántos años distintos (anio) hay representados?
df['anio'].nunique()

17

Hay un total de $17$ años distintos representados.

In [168]:
#¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?
df.duplicated().sum()

np.int64(0)

No existen filas duplicadas, por tanto hay una cantidad de $0$ filas duplicadas.

In [169]:
#Validación cruzada de columnas
#¿Hay inconsistencias entre el país (pais) y su código (codigo_iso)? (por ejemplo, un mismo código ISO asociado a más de un país)
inconsistencias= df.groupby('codigo_iso')['pais'].nunique()
inconsistencias[inconsistencias > 1]

,pais
codigo_iso,


No existen inconsistencias dentre el pais y su codigo_iso.




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [170]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)]


In [171]:
#a)
años = sorted(df_america['anio'].unique())
for i in años:
    df_year = df_america[df_america['anio'] == i].dropna(subset=['indice'])
    if df_year.empty:
        continue  # salta el año si no tiene datos, da error si no es asi
    pais_min = df_year.loc[df_year['indice'].idxmin()]
    pais_max = df_year.loc[df_year['indice'].idxmax()]
    print("Año: ", i)
    print("País con menor indice: ", pais_min['pais'])
    print("País con mayor indice: ", pais_max['pais'])

Año:  2001
País con menor indice:  Canadá
País con mayor indice:  Cuba
Año:  2002
País con menor indice:  Trinidad y Tobago
País con mayor indice:  Cuba
Año:  2003
País con menor indice:  Trinidad y Tobago
País con mayor indice:  Argentina
Año:  2004
País con menor indice:  Trinidad y Tobago
País con mayor indice:  Cuba
Año:  2005
País con menor indice:  Bolivia
País con mayor indice:  Cuba
Año:  2006
País con menor indice:  Canadá
País con mayor indice:  Cuba
Año:  2007
País con menor indice:  Canadá
País con mayor indice:  Cuba
Año:  2008
País con menor indice:  Canadá
País con mayor indice:  Cuba
Año:  2009
País con menor indice:  Estados Unidos
País con mayor indice:  Cuba
Año:  2012
País con menor indice:  Jamaica
País con mayor indice:  Cuba
Año:  2013
País con menor indice:  Jamaica
País con mayor indice:  Cuba
Año:  2014
País con menor indice:  Canadá
País con mayor indice:  Cuba
Año:  2015
País con menor indice:  Costa Rica
País con mayor indice:  Cuba
Año:  2017
País con meno

In [172]:
#b)
df_america_valid = df_america.dropna(subset=['indice'])
minv=df_america_valid.loc[df_america_valid.groupby('anio')['indice'].idxmin()]
maxv=df_america_valid.loc[df_america_valid.groupby('anio')['indice'].idxmax()]
print("Pais con menor indice: ")
print(df_america_valid.loc[df_america_valid.groupby('anio')['indice'].idxmin()])
print("Pais con mayor indice: ")
print(df_america_valid.loc[df_america_valid.groupby('anio')['indice'].idxmax()])

Pais con menor indice: 
     codigo_iso  anio  indice  ranking               pais
27          CAN  2001    0.80      2.0             Canadá
341         TTO  2002    1.00      2.0  Trinidad y Tobago
520         TTO  2003    2.00     30.0  Trinidad y Tobago
699         TTO  2004    2.00     31.0  Trinidad y Tobago
737         BOL  2005    4.50     63.0            Bolivia
922         CAN  2006    4.88     84.0             Canadá
1101        CAN  2007    3.33     50.0             Canadá
1280        CAN  2008    3.70     62.0             Canadá
1602        USA  2009    6.75    115.0     Estados Unidos
1692        JAM  2012    9.88    176.0            Jamaica
1871        JAM  2013   10.90      6.0            Jamaica
1996        CAN  2014   10.99      3.0             Canadá
2186        CRI  2015   11.10      2.0         Costa Rica
2544        CRI  2017   11.93      6.0         Costa Rica
2766        JAM  2018   11.33      6.0            Jamaica
2945        JAM  2019   11.13      8.0          

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [173]:
#Tarea principal
#Construye una tabla dinámica (pivot_table) donde las filas correspondan a los países, las columnas a los años (anio) y los valores sean el indice máximo alcanzado por cada país en ese año.
pivot=pd.pivot_table(df_america_valid, values='indice', index='pais', columns='anio', aggfunc='max', fill_value=0)
display(pivot)

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Antigua y Barbuda,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.81,0.00,0.00,0.00,0.00,0.00
Argentina,12.0,15.17,35826.00,13.67,17.30,24.83,14.08,11.33,16.35,25.67,25.27,26.11,25.09,25.07,26.05,28.30
Belize,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.05,18.54,20.61,23.43,24.55,27.50
Bolivia,14.5,9.67,20.00,9.67,4.50,21.50,28.20,24.17,28.13,32.80,31.04,31.29,31.78,33.88,32.45,35.38
Brasil,18.8,16.75,16.50,14.50,17.17,25.25,18.00,15.88,16.60,32.75,34.03,31.93,32.62,33.58,31.20,32.79
Canadá,0.8,1.83,3.33,4.50,4.50,4.88,3.33,3.70,7.00,12.69,10.99,10.99,15.26,16.53,15.28,15.69
Chile,6.5,6.83,10.00,11.75,11.63,12.13,11.50,10.50,10.50,26.24,25.80,23.00,19.23,20.53,22.69,25.65
Colombia,40.8,49.17,47.38,40.17,44.75,42.33,35.50,40.13,51.50,37.48,36.68,39.08,44.11,41.47,41.03,42.82
Costa Rica,4.3,3.83,7.63,8.50,6.67,6.50,5.10,8.00,8.08,12.08,12.23,12.26,11.10,11.93,14.01,12.24


In [174]:
#Preguntas adicionales
#¿Qué país tiene el mayor valor de indice en toda la tabla resultante?
max=df_america_valid['indice'].max()
df_america_valid[df_america_valid['indice']==max]

,codigo_iso,anio,indice,ranking,pais
363,ARG,2003,35826.0,35.0,Argentina


In [175]:
#¿Y cuál tiene el menor (distinto de cero)?
min=df_america_valid['indice'].min()
df_america_valid[df_america_valid['indice']==min]

,codigo_iso,anio,indice,ranking,pais
27,CAN,2001,0.8,2.0,Canadá


In [176]:
#¿Qué años presentan en promedio los valores de indice más altos?
pd.pivot_table(df_america_valid, values='indice', index='pais', columns='anio', aggfunc='mean').mean(axis=0).sort_values(ascending=False)

,0
anio,
2003,1451.944000
2019,30.802963
2017,30.164444
2018,29.781852
2015,29.654074
2014,28.733333
2012,28.526923
2013,27.958929
2006,25.126667


Los indices mas altos en promedio son los años $2003$, $2019$ y $2017$.\
Los indices mas bajos en promedio son los años $2001$ y $2002$.

In [177]:
#¿Qué país muestra mayor variabilidad (diferencia entre su máximo y mínimo indice a lo largo del tiempo)?
pivot.max(axis=1) - pivot.min(axis=1)

,0
pais,
Antigua y Barbuda,20.81
Argentina,35814.67
Belize,27.50
Bolivia,30.88
Brasil,19.53
Canadá,15.73
Chile,19.74
Colombia,16.00
Costa Rica,10.18


Argentina presenta una mayor diferencia entre su minimo y maximo indice a lo largo del tiempo.

In [178]:
#¿Existen países con índice constante a lo largo de todos los años registrados?
pivot[(pivot.max(axis=1) - pivot.min(axis=1)) == 0]


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


No existe pais con indice constante.

In [179]:
# ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?
pivot[(pivot == 0).all(axis=1)]

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


Todos los paises tiene al menos un dato.